### Import Libraries

In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from dataset import *
from PIL import Image
import json
import time

import torch
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel
from transformers import AutoProcessor, BitsAndBytesConfig, AutoModelForImageTextToText
from datasets import Dataset
import re
from Levenshtein import distance as edit_distance

In [ ]:
import gc
import time


def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

### Prepare Training Data

In [3]:
model_id = "HuggingFaceTB/SmolVLM2-2.2B-Instruct"

In [4]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in extracting visual data.
Your task is to extract from the image the requested fields and return the result only in a valid JSON format."""

In [26]:
processor = AutoProcessor.from_pretrained(model_id, use_fast=True, padding_side="left")

In [ ]:
imgs_fn = []

In [ ]:
def format_data(sample):
    imgs_fn.append(sample.image_path.split("/")[-1])
    pil_image = Image.open(sample.image_path)

    field_names = set(sorted([entity.label for entity in sample.entities]))
    output_format = json.dumps({field: ".." for field in field_names})

    prompt = "Extract the following {fields} from the above document. \
              If a field is not present, return ''. Return the output in a valid JSON format like {output_format}. \
              For date fields make sure to return date in this format dd/mm/yyyy". \
        .format(
            fields = list(field_names),
            output_format = output_format
        )

    conversation = [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": system_message}
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    return {
        "prompt": prompt,
        "image": pil_image,
        "answer": json.dumps(sample.to_json("kie"))
    }


In [ ]:
train_dataset = [format_data(sample) for sample in SROIE(tasks=["kie"], split="train")]
test_dataset = [format_data(sample) for sample in SROIE(tasks=["kie"], split="test")]

print(train_dataset)

Dataset({
    features: ['prompt', 'image', 'answer'],
    num_rows: 626
})


In [33]:
print(train_dataset[0]["prompt"])
print(train_dataset[0]["image"])
print(train_dataset[0]["answer"])

<|im_start|>System: <end_of_utterance>
User:<image>Extract the following ['company', 'address', 'total', 'date'] from the image<end_of_utterance>
Assistant:
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=463x1013 at 0x7C4B68B265D0>
{"company": "BOOK TA .K (TAMAN DAYA) SDN BHD", "date": "25/12/2018", "address": "NO.53 55,57 & 59, JALAN SAGU 18, TAMAN DAYA, 81100 JOHOR BAHRU, JOHOR.", "total": "9.00"}


### Training Pipeline

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    _attn_implementation="eager",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
model = PeftModel.from_pretrained(
    model = model,
    model_id = "final",
    is_trainable = True
)

In [ ]:
USE_LORA = False
USE_QLORA = True

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj'],
    #init_lora_weights="gaussian"
)

In [ ]:
#model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    output_dir="result/smolvlm2-grpo",
    learning_rate=2e-6,
    remove_unused_columns=False,  # to access the solution column in accuracy_reward
    #num_train_epochs=1,
    max_steps=800,
    bf16=True,
    # Parameters that control the data preprocessing
    per_device_train_batch_size=1,
    max_completion_length=3000,  # default: 256
    num_generations=4,  # default: 8
    generation_batch_size=4,
    gradient_accumulation_steps=4,
    max_prompt_length=None,
    # Parameters related to reporting and saving
    report_to="none",
    logging_steps=100,
    push_to_hub=False,
    save_strategy="steps",
    save_steps=100,
    use_liger_loss=True
)

In [ ]:
def json_parsable(completions, **kwargs) -> list[float]:
    responses = [completion for completion in completions]
    results = []
    for response in responses:
        try:
            parsed = json.loads(re.search(r"```json\n\s*(.*?)\s*\n```", response, re.DOTALL).group(1).strip())
            if isinstance(parsed, dict):
                results.append(4.0)
            else:
                results.append(-4.0)
        except:
            results.append(-4.0)
    print("json_parsable: ", results)
    return results

def all_fields_present(completions, answer, **kwargs) -> list[float]:
    responses = [completion for completion in completions]
    results = []
    for response, gt in zip(responses, answer):
        labels = json.loads(gt).keys()
        score = 0.0
        try:
            response_dict = json.loads(re.search(r"```json\n\s*(.*?)\s*\n```", response, re.DOTALL).group(1).strip())
            if isinstance(response_dict, dict):
                for label in list(labels):
                    if label in response_dict.keys():
                        score += 2.0
                    else:
                        score += -2.0

                #results.append(2.0 if labels.issubset(response_dict.keys()) else -0.5)
                results.append(score)
            else:    
                results.append(-4.0)
        except:
            results.append(-4.0)
    print("fields_present: ", results)
    return results

def correct_labels(completions, answer, **kwargs) -> list[float]:
    results = []
    for completion, gt in zip(completions, answer):
        score = 0.0
        labels = json.loads(gt).keys()
        try:
            completion_dict = json.loads(re.search(r"```json\n\s*(.*?)\s*\n```", completion, re.DOTALL).group(1).strip())
            gt_dict = json.loads(gt)
            for label in labels:
                pred = completion_dict.get(label, None)
                gt_value = gt_dict.get(label, None)

                if pred == None:
                    score += -2.0
                else:
                    dist = edit_distance(pred, gt_value)
                    if dist < 2:
                        score += 2.0
                    else:
                        score += -dist
                
                '''
                if pred == gt_value and pred is not None:
                    score += 1.0
                else:
                    score -= 2.0
                '''
            results.append(score)
        except:
            results.append(-4.0)
    print("corrent labels: ",results)
    return results

# TODO: aggiungere funzione per calcolare editdistance sui campi

In [ ]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model,
    processing_class=processor,
    reward_funcs=[
        json_parsable,
        all_fields_present,
        correct_labels
    ],
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(f"{training_args.output_dir}/final")

In [ ]:
imgs_fn = []

test_dataset = [format_data(sample) for sample in DocILE(tasks=["kie"], split="val")]
print(test_dataset[0])

In [ ]:
clear_memory()

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    _attn_implementation="eager",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

processor = AutoProcessor.from_pretrained(model_id)

model.load_adapter(f"result-docile/smolvlm2-grpo/final")

In [ ]:
results = {}

for data, fn in zip(test_dataset, imgs_fn):
    inputs = processor.apply_chat_template(
        data["prompt"],
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to("cuda", dtype=torch.bfloat16)

    start = time.time()
    generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=1000)
    end = time.time()

    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )
    results[fn] = dict(
        response = generated_texts[0],
        inference_time = end - start
    )

    with open(f"result-docile/smolvlm2-grpo.json", "w") as f:
        json.dump(results, f, indent=4)